In [1]:
import requests
import pandas as pd
import time
import numpy as np
import re
import sys
import random

In [2]:
jeju_stay =  pd.read_csv('../data/jeju_stay_modified.csv')

In [3]:
jeju_stay.shape

(933, 25)

In [4]:
# 1차 검색 제주 000 
url = 'https://map.naver.com/v5/api/search'
params = {
    'calller': 'pcweb',
    'query': '',
    'type': 'all',
    'searchCoord': '126.56379330000001;33.24652849999992',
    'page': 1,
    'displayCount': 20,
    'isPlaceRecommendationReplace': True,
    'lang': 'ko'
}
tmp_dict = dict()
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; csrf_token=b22420dac145f9757a431121f5b4c25948818847853c526728a4d2219155c97e9b6490d088524f5e2d1f9fdd189f23d95bdd86f6e9ca8c8b07bf30c857e040c6; page_uid=d78aa3bb-613a-4eab-8035-b9c4712fa1de',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}
params2 ={
    'lang' :'ko'
}
#n차 검색 제주도 주소 검색 후 해당 위치의 가게 정보를 조회 
url3 = 'https://map.naver.com/v5/api/addresses/place/'
params3= {
    'address':'',
    'lang':'ko',
    'page':1
}
headers2 = {
    'authority': 'map.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; page_uid=hXuWqwp0YiRssBM4BUGssssssMo-163853; BMR=s=1658907970093&r=https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=heygun&logNo=221520454294&r2=https://www.google.com/; page_uid=3c7d105d-de2b-4c43-8a74-1c2abebf354d',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma ': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

In [5]:
title = jeju_stay['title']

In [6]:
title.tail()

928         민박바당집
929    리니민박(RINY)
930          코쿤호텔
931      코업시티호텔성산
932         올레리조트
Name: title, dtype: object

In [8]:
stay_result = []
none_lst = []
for num,v in enumerate(title):
    searchword = '제주 '+ title[num]
    #정규표현식으로 ( ) 안 텍스트 전부 삭제
    rmve_bracket = "\(.*\)|\s-\s.*"
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(2)
    response1 = requests.get(url,params,headers=headers)
    if response1.status_code == 200:
        j_text = response1.json()
    else:
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
        
    #장소정보 저장    
    tmp_places = j_text['result']['place']    
    search_bool = False
    #검색결과 없을 때
    if tmp_places is None:
        none_lst.append(title[num])
        print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
        continue
    #검색 결과 존재시    
    else:
        tmp_list=tmp_places['list']
        #검색결과가 하나일 때 바로 조회
        if len(tmp_list) == 1:
            tmp_id = tmp_list[0]['id']
            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
            time.sleep(2)
            response2 = requests.get(url2,params2,headers=headers2)
            if response2.status_code == 200:
                tmp_dict = response2.json()
                stay_result.append(tmp_dict)
                search_bool = True
                print(f'{num}번째 {searchword} 중 {tmp_id}의 여행지 요약본 저장 완료')
            else:
                print(f'{num}번째 {searchword}의 {tmp_id}로 여행지 요약본 가져오기 실패')
                sys.exit('response2.status_code 조회해보기')
            continue
        #검색결과 여러개일 때    
        else:
            for index, lst in enumerate(tmp_list):
                tmp_categories =lst['category']
                for category in tmp_categories:
                    if '숙박' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            stay_result.append(tmp_dict)
                            search_bool = True
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 여행지 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 여행지 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '호텔' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            stay_result.append(tmp_dict)
                            search_bool = True
                            print(f'{num}번째 {searchword} 중 {tmp_id}의  기념품 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 기념품 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
            
        if search_bool is False:
            none_lst.append(title[num])
            print(f'{num}번째 {searchword}중 숙박업소 존재X ')
                    
                        
    print(f'{num}번째 {searchword} 종료')

0번째 제주 하이제주호텔 중 1859085120의 여행지 요약본 저장 완료
0번째 제주 하이제주호텔 중 1059410525의 여행지 요약본 저장 완료
0번째 제주 하이제주호텔 중 1024729534의 여행지 요약본 저장 완료
0번째 제주 하이제주호텔 종료
1번째 제주 하워드존슨 제주 연동 호텔 중 1261120776의 여행지 요약본 저장 완료
2번째 제주 신화관 제주신화월드 호텔 앤 리조트 중 1101198197의 여행지 요약본 저장 완료
3번째 제주  메리어트관 제주신화월드 호텔앤리조트 중 1513281659의 여행지 요약본 저장 완료
4 번째 서머셋 제주신화월드 호텔앤리조트 조회 결과 존재 X
5번째 제주 랜딩관 제주신화월드 호텔앤리조트 중 1497292200의 여행지 요약본 저장 완료
6번째 제주 야스라기펜션 중 13565703의 여행지 요약본 저장 완료
7번째 제주 대명리조트 제주 중 12896850의 여행지 요약본 저장 완료
7번째 제주 대명리조트 제주 종료
8번째 제주 외돌개나라 중 13466999의 여행지 요약본 저장 완료
9번째 제주 까사데비발디호텔 중 38737405의 여행지 요약본 저장 완료
10 번째 이찌마루&잠비 조회 결과 존재 X
11번째 제주 오시록헌펜션 중 1433255121의 여행지 요약본 저장 완료
11번째 제주 오시록헌펜션 중 1591011949의 여행지 요약본 저장 완료
11번째 제주 오시록헌펜션 종료
12번째 제주 꿀잠자리 중 37475183의 여행지 요약본 저장 완료
13번째 제주 피디스테이션 중 21667025의 여행지 요약본 저장 완료
14번째 제주 달집게스트하우스 중 1537437915의 여행지 요약본 저장 완료
15 번째 UncleBORO게스트하우스 조회 결과 존재 X
16번째 제주 숨비아일랜드 중 1217846074의 여행지 요약본 저장 완료
17번째 제주 게으른소나기 중 30847223의 여행지 요약본 저장 완료
18번째 제주 노을과어울림 중 33704771의 여행지 요약본 저장 완료
18번째 제주 노을과어울림 

127번째 제주 퐁낭하우스제주 중 59480093의 여행지 요약본 저장 완료
127번째 제주 퐁낭하우스제주 중 1785001322의 여행지 요약본 저장 완료
127번째 제주 퐁낭하우스제주 중 1178793977의 여행지 요약본 저장 완료
127번째 제주 퐁낭하우스제주 중 1110186089의 여행지 요약본 저장 완료
127번째 제주 퐁낭하우스제주 중 105581345의 여행지 요약본 저장 완료
127번째 제주 퐁낭하우스제주 종료
128 번째 For 安(포안) 조회 결과 존재 X
129번째 제주 쉴만한 물가 중 1204572074의 여행지 요약본 저장 완료
129번째 제주 쉴만한 물가 중 1852466345의 여행지 요약본 저장 완료
129번째 제주 쉴만한 물가 종료
130 번째 제주스웨덴리조트펜션 조회 결과 존재 X
131번째 제주 캐슬 드 한림 중 37912298의 여행지 요약본 저장 완료
132번째 제주 코코스펜션 중 11795849의 여행지 요약본 저장 완료
133번째 제주 해피데이 중 35873333의 여행지 요약본 저장 완료
133번째 제주 해피데이 중 11578399의 여행지 요약본 저장 완료
133번째 제주 해피데이 중 37005941의 여행지 요약본 저장 완료
133번째 제주 해피데이 종료
134번째 제주 해피데이 민박 중 11578399의 여행지 요약본 저장 완료
135번째 제주 카이리조트 중 31699389의 여행지 요약본 저장 완료
135번째 제주 카이리조트 중 1989636409의 여행지 요약본 저장 완료
135번째 제주 카이리조트 종료
136번째 제주 초록하우스 중 1561444200의 여행지 요약본 저장 완료
136번째 제주 초록하우스 중 35228696의 여행지 요약본 저장 완료
136번째 제주 초록하우스 중 36947581의 여행지 요약본 저장 완료
136번째 제주 초록하우스 종료
137번째 제주 라신비 중 975477104의 여행지 요약본 저장 완료
137번째 제주 라신비 종료
138번째 제주 플라마 펜션 중 1768726578의

230 번째 해안길펜션 조회 결과 존재 X
231번째 제주 드헤브펜션 중 38460416의 여행지 요약본 저장 완료
232번째 제주 제주휴리조트 중 33855950의 여행지 요약본 저장 완료
232번째 제주 제주휴리조트 종료
233번째 제주 아모르하우스 중 33961192의 여행지 요약본 저장 완료
233번째 제주 아모르하우스 종료
234번째 제주 우도올레펜션 중 11727821의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 36745154의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 34177419의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1567976981의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 16892664의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 17041833의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1958752311의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1519847948의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1007980616의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 19455357의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 37700685의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 37171863의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1356633434의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 16892944의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1984422668의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 16892648의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1132565151의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 38549072의 여행지 요약본 저장 완료
234번째 제주 우도올레펜션 중 1842451513의 여행지 요약본 저장 완료

336번째 제주 호텔더블유 탑동점 중 33938009의 여행지 요약본 저장 완료
337번째 제주 수상한집 광보네 중 1995811910의 여행지 요약본 저장 완료
338번째 제주 관음사 야영장 중 13542674의 여행지 요약본 저장 완료
338번째 제주 관음사 야영장 종료
339번째 제주 펜션연리 중 37198403의 여행지 요약본 저장 완료
339번째 제주 펜션연리 종료
340번째 제주 호스텔린든 중 1937798504의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 36460705의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 1036837689의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 16767068의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 1936118945의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 1487731173의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 1692600120의 여행지 요약본 저장 완료
341번째 제주 바다향기 중 35563647의 여행지 요약본 저장 완료
341번째 제주 바다향기 종료
342 번째 오렌지트리 호텔 조회 결과 존재 X
343번째 제주 씨제이 관광호텔 중 34314798의 여행지 요약본 저장 완료
344 번째 비지터게스트하우스 조회 결과 존재 X
345번째 제주 라헨느리조트 중 16994295의 여행지 요약본 저장 완료
345번째 제주 라헨느리조트 종료
346번째 제주 호텔레오 중 35603475의 여행지 요약본 저장 완료
346번째 제주 호텔레오 종료
347번째 제주 제주인호텔 중 1131494103의 여행지 요약본 저장 완료
347번째 제주 제주인호텔 종료
348번째 제주 제주팔레스호텔 중 11583206의 여행지 요약본 저장 완료
348번째 제주 제주팔레스호텔 종료
349 번째 신강남모텔 조회 결과 존재 X
350번째 제주 유앤아이게스트하우스 중 16714287의 여행지 요약본 저장 완료
351번째 제주 제주마실게스트하우스 중 32848730

436번째 제주 호텔 앨리스앤트렁크 중 1302453180의 여행지 요약본 저장 완료
437번째 제주 바다추억 중 10976116의 여행지 요약본 저장 완료
437번째 제주 바다추억 종료
438번째 제주 추자바다25시 중 11582176의 여행지 요약본 저장 완료
439번째 제주 아뜨네통나무펜션 중 11781998의 여행지 요약본 저장 완료
440번째 제주 물돌이민박 중 11569157의 여행지 요약본 저장 완료
441번째 제주 제주썬호텔&카지노 중 38278713의 여행지 요약본 저장 완료
441번째 제주 제주썬호텔&카지노 종료
442번째 제주 유니호텔제주 중 36281072의 여행지 요약본 저장 완료
442번째 제주 유니호텔제주 종료
443번째 제주 그라벨호텔제주 중 35394656의 여행지 요약본 저장 완료
443번째 제주 그라벨호텔제주 종료
444번째 제주 블랙샌즈호텔 중 36857677의 여행지 요약본 저장 완료
445번째 제주 제이드림호텔 중 1547999298의 여행지 요약본 저장 완료
446번째 제주 약천골민박 중 13449602의 여행지 요약본 저장 완료
447번째 제주 켄싱턴리조트 서귀포 중 1269816425의 여행지 요약본 저장 완료
447번째 제주 켄싱턴리조트 서귀포 중 11628464의 여행지 요약본 저장 완료
447번째 제주 켄싱턴리조트 서귀포 중 11627290의 여행지 요약본 저장 완료
447번째 제주 켄싱턴리조트 서귀포 종료
448번째 제주 제주아이브리조트 중 32581738의 여행지 요약본 저장 완료
448번째 제주 제주아이브리조트 중 675773040의 여행지 요약본 저장 완료
448번째 제주 제주아이브리조트 중 760973286의 여행지 요약본 저장 완료
448번째 제주 제주아이브리조트 중 811434753의 여행지 요약본 저장 완료
448번째 제주 제주아이브리조트 중 791494472의 여행지 요약본 저장 완료
448번째 제주 제주아이브리조트 중 264782411의 여행지 요약본 저장 완료
448번째 제주 제주아이브리조트

537번째 제주 소담펜션 중 38502427의 여행지 요약본 저장 완료
537번째 제주 소담펜션 중 1730384947의 여행지 요약본 저장 완료
537번째 제주 소담펜션 중 1067614940의 여행지 요약본 저장 완료
537번째 제주 소담펜션 종료
538번째 제주 그린사이드펜션 중 11782839의 여행지 요약본 저장 완료
539번째 제주 호텔빠레브 중 32876520의 여행지 요약본 저장 완료
539번째 제주 호텔빠레브 중 1142469850의 여행지 요약본 저장 완료
539번째 제주 호텔빠레브 종료
540번째 제주 토스카나호텔 중 33093640의 여행지 요약본 저장 완료
540번째 제주 토스카나호텔 종료
541번째 제주 신라호텔제주 중 11583210의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 중 36183644의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 중 32594202의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 중 20243581의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 중 1068368011의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 중 37695797의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 중 16843529의 여행지 요약본 저장 완료
541번째 제주 신라호텔제주 종료
542번째 제주 씨에스호텔앤리조트 중 11588707의 여행지 요약본 저장 완료
543번째 제주 라마다 앙코르 서귀포 이스트 호텔 중 1744448811의 여행지 요약본 저장 완료
544번째 제주 취다선리조트 중 1942640444의 여행지 요약본 저장 완료
544번째 제주 취다선리조트 종료
545번째 제주 호텔서귀피안 중 1114138745의 여행지 요약본 저장 완료
545번째 제주 호텔서귀피안 종료
546번째 제주 셋째날 중 20028930의 여행지 요약본 저장 완료
547번째 제주 헤세드하우스 중 36281054의 여행지 요약본 저장 완료
548번째 제주 제주펜션향림원 중 10987621의 여

628번째 제주 꿈꾸는바다 중 11782065의 여행지 요약본 저장 완료
628번째 제주 꿈꾸는바다 종료
629번째 제주 마니주팬션 중 31549538의 여행지 요약본 저장 완료
630 번째 예다움휴양펜션 조회 결과 존재 X
631번째 제주 소풍이야기 펜션 중 33109149의 여행지 요약본 저장 완료
632번째 제주 이어도펜션 중 11782810의 여행지 요약본 저장 완료
633번째 제주 바다풍경펜션 중 11831769의 여행지 요약본 저장 완료
634번째 제주 아모렉스리조트 중 13073181의 여행지 요약본 저장 완료
634번째 제주 아모렉스리조트 종료
635번째 제주 제주쉼표 중 11064997의 여행지 요약본 저장 완료
635번째 제주 제주쉼표 중 1953035660의 여행지 요약본 저장 완료
635번째 제주 제주쉼표 중 1274188482의 여행지 요약본 저장 완료
635번째 제주 제주쉼표 중 1806613631의 여행지 요약본 저장 완료
635번째 제주 제주쉼표 중 1248156642의 여행지 요약본 저장 완료
635번째 제주 제주쉼표 종료
636번째 제주 리버풀펜션 중 16809951의 여행지 요약본 저장 완료
636번째 제주 리버풀펜션 중 1044344829의 여행지 요약본 저장 완료
636번째 제주 리버풀펜션 종료
637번째 제주 아스타호텔 중 38766304의 여행지 요약본 저장 완료
637번째 제주 아스타호텔 종료
638번째 제주 소랑풀빌라 중 821672337의 여행지 요약본 저장 완료
638번째 제주 소랑풀빌라 중 1938354328의 여행지 요약본 저장 완료
638번째 제주 소랑풀빌라 종료
639번째 제주 스토리하우스펜션 중 20289666의 여행지 요약본 저장 완료
639번째 제주 스토리하우스펜션 중 1236590173의 여행지 요약본 저장 완료
639번째 제주 스토리하우스펜션 종료
640번째 제주 제주목화휴양펜션 중 11778969의 여행지 요약본 저장 완료
641번째 제주 펄리플러스 중 37223268의 여행지 요약본 저장 완료

720번째 제주 돌담민박 중 569188212의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1082532294의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1732599010의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 289137315의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1863030148의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1797331307의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1812188001의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1992445952의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1052144028의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1601584230의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1607465242의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1642314956의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1639592593의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1622440016의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1519268854의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1055485837의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1662008863의 여행지 요약본 저장 완료
720번째 제주 돌담민박 중 1684685650의 여행지 요약본 저장 완료
720번째 제주 돌담민박 종료
721번째 제주 아올키즈펜션 중 32794378의 여행지 요약본 저장 완료
722번째 제주 느르왓 중 34301123의 여행지 요약본 저장 완료
723번째 제주 바당정원펜션 중 1434228660의 여행지 요약본 저장 완료
724 번째 제주도푸른산푸른바다 조회 결과 존재 X
725 번째 리치망고펜션 조회 결과 존재 X
726번째 제주 빌림 중 36831059의 여행지 요약본 저장 완료
726번째 제주 빌림 

807번째 제주 우도사랑 중 17041940의 여행지 요약본 저장 완료
807번째 제주 우도사랑 중 11682034의 여행지 요약본 저장 완료
807번째 제주 우도사랑 종료
808번째 제주 이레민박 중 1140073332의 여행지 요약본 저장 완료
808번째 제주 이레민박 종료
809번째 제주 나무아래독채펜션 중 38554859의 여행지 요약본 저장 완료
810번째 제주 볼몽지엥하우스 중 36060598의 여행지 요약본 저장 완료
811번째 제주 씨앤하우스 중 32755230의 여행지 요약본 저장 완료
811번째 제주 씨앤하우스 종료
812번째 제주 레몬이네 민박 중 1223678507의 여행지 요약본 저장 완료
813번째 제주 콴도제주 중 469727183의 여행지 요약본 저장 완료
814번째 제주 구름밭하우스 중 1689477707의 여행지 요약본 저장 완료
815번째 제주 부르네스테이 중 1571584361의 여행지 요약본 저장 완료
816번째 제주 언니네여인숙 중 98819234의 여행지 요약본 저장 완료
817번째 제주 하랑 게스트하우스 중 35142645의 여행지 요약본 저장 완료
818번째 제주 성게돌 펜션  중 788648580의 여행지 요약본 저장 완료
819번째 제주 언덕위에 빨간지붕 중 1466643231의 여행지 요약본 저장 완료
820번째 제주 또올레펜션&게스트하우스 중 33680136의 여행지 요약본 저장 완료
821번째 제주 나무사이로햇살이 중 37769727의 여행지 요약본 저장 완료
822번째 제주 세화맨션 중 36224842의 여행지 요약본 저장 완료
823번째 제주 발리인제주 중 1352349577의 여행지 요약본 저장 완료
824번째 제주 포비네오두막게스트하우스 중 35394835의 여행지 요약본 저장 완료
825번째 제주 달콤한아침 중 36302157의 여행지 요약본 저장 완료
826번째 제주 레프트핸더 게스트하우스 중 1769616085의 여행지 요약본 저장 완료
827번째 제주 도시락게스트하우스 중 34069363의 여

## 이름으로 못가져온정보들 주소로 크롤링

In [15]:
tmp_df = pd.DataFrame(stay_result)

In [16]:
tmp_df.head()

,id,isSite,name,type,rCode,x,y,posExact,address,fullAddress,...,michelinGuide,broadcastInfo,marker,markerSelected,datalab,reviewCount,dynamicData,markerLabel,isParkingSupported,indoor
0,1859085120,True,Hi제주호텔,drt,1411025031,126.253788,33.391936,True,제주 제주시 한림읍 협재리 1165,제주특별자치도 제주시 한림읍 협재리 1165,...,None,None,nres://marker/00079-00087,nres://marker/00079-00251,None,13,"{'pension': {'title': 'Hi제주호텔', 'roomMinPrice'...","{'text': None, 'style': 'default'}",False,NaN
1,1059410525,True,하이레지던스,drt,14110137,126.490975,33.489440,True,제주 제주시 연동 272-22,제주특별자치도 제주시 연동 272-22,...,None,None,nres://marker/00079-00087,nres://marker/00079-00251,None,229,"{'pension': {'title': '하이레지던스', 'roomMinPrice'...","{'text': None, 'style': 'default'}",False,NaN
2,1024729534,True,하이프로,drt,14130101,126.562457,33.247490,True,제주 서귀포시 서귀동,제주특별자치도 서귀포시 서귀동,...,None,None,nres://marker/00076-00087,nres://marker/00076-00251,None,0,"{'pension': {'title': '하이프로', 'roomMinPrice': ...","{'text': None, 'style': 'default'}",False,NaN
3,1261120776,True,글로스터호텔 제주,drt,14110137,126.492632,33.490087,True,제주 제주시 연동 281-22,제주특별자치도 제주시 연동 281-22,...,None,None,nres://marker/00079-00087,nres://marker/00079-00251,None,180,"{'pension': {'title': '글로스터호텔 제주', 'roomMinPri...","{'text': None, 'style': 'default'}",False,NaN
4,1101198197,True,신화관 제주신화월드 호텔앤리조트,drt,1413031027,126.315878,33.306695,True,제주 서귀포시 안덕면 서광리 산24,제주특별자치도 서귀포시 안덕면 서광리 산24,...,None,None,nres://marker/00079-00087,nres://marker/00079-00251,None,818,"{'pension': {'title': '신화관 제주신화월드 호텔앤리조트', 'ro...","{'text': None, 'style': 'default'}",False,NaN


In [10]:
none_df =jeju_stay[jeju_stay['title'].isin(none_lst)]

In [12]:
none_df['roadaddress'].isnull().sum()

0

In [19]:
none_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 4 to 929
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   alltag           103 non-null    object 
 1   contentsid       108 non-null    object 
 2   contentscd       108 non-null    object 
 3   title            108 non-null    object 
 4   address          108 non-null    object 
 5   roadaddress      108 non-null    object 
 6   tag              108 non-null    object 
 7   introduction     108 non-null    object 
 8   readcnt          108 non-null    int64  
 9   likecnt          108 non-null    int64  
 10  reviewcnt        108 non-null    int64  
 11  markcnt          108 non-null    int64  
 12  snssharecnt      108 non-null    int64  
 13  schedulecnt      108 non-null    int64  
 14  visitcnt         108 non-null    int64  
 15  evelpt           38 non-null     float64
 16  latitude         108 non-null    float64
 17  longitude       

In [14]:
for v in none_df['roadaddress']:
    print(v)

제주특별자치도 서귀포시 안덕면 신화역사로304번길 89
제주특별자치도 제주시 구좌읍 월정1길 54-11
제주특별자치도 제주시 한경면 저지12길 60-6
제주특별자치도 제주시 용마로4길 15
제주특별자치도 제주시 한경면 신창상동길 20-1
제주특별자치도 제주시 구좌읍 하도15길 153-5
제주특별자치도 제주시 구좌읍 평대4길 20-1
제주특별자치도 서귀포시 안덕면 서광서문로 55-7
제주특별자치도 제주시 관덕로13길 11
제주특별자치도 서귀포시 안덕면 난드르로21번길 13-7
제주특별자치도 서귀포시 남원읍 서성로 459
제주특별자치도 제주시 애월읍 유수암동3길 45
제주특별자치도 제주시 애월읍 하소로 769-15
제주특별자치도 제주시 애월읍 천덕로 313-3
제주도 제주시 애월읍 애월해안로 384-23
제주특별자치도 제주시 한림읍 옹포2길 12
제주특별자치도 제주시 한림읍 명재로 100-45
제주특별자치도 제주시 한림읍 금능남로 127
제주특별자치도 서귀포시 안덕면 쇠물도로 14
제주특별자치도 서귀포시 안덕면 형제해안로 24
제주특별자치도 제주시 애월읍 화전길 201
제주특별자치도 서귀포시 표선면 민속해안로 381
제주도 서귀포시 표선면 표선동서로 139
제주특별자치도 서귀포시 표선면 표선당포로 27
제주특별자치도 서귀포시 성산읍 신양로122번길 4-6
제주특별자치도 제주시 우도면 우도해안길 690
제주특별자치도 서귀포시 성산읍 시흥상동로68번길 26-1
제주특별자치도 서귀포시 동홍로262번길 62
제주특별자치도 제주시 구좌읍 일주동로 1626
제주특별자치도 서귀포시 남원읍 남조로 861-83
제주특별자치도 서귀포시 남원읍 위미중앙로196번길 6-12
제주특별자치도 서귀포시 솔동산로22번길 20
제주특별자치도 제주시 중앙로12길 5
제주특별자치도 제주시 관덕로8길 7-12
제주특별자치도 제주시 오현길 85
제주특별자치도 제주시 서광로3길 2
제주특별자치도 제주시 은남4길 42
제주특별자치도 제주시 애월읍 중산간서로 5409
제주특별자치도 제주시 은남1길 8
제주

In [20]:
none_lst2 = []

In [22]:
for i,v in enumerate(none_df['roadaddress']):
    searchword = v
    #정규표현식으로 0000구 000로 (00동)형식에서 (00) 삭제
    rmve_bracket = "\(.*\)|\s-\s.*"
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(random.uniform(1.5,2.5))
    response1 = requests.get(url,params,headers=headers2)
    if response1.status_code == 200:
        j_text = response1.json()
    else: 
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
    address = j_text['result']['address']
    if address is None:
        none_lst2.append(v)
        print(f'{v} 검색결과는 존재하지않음')
    else:
        roadAddress = address['roadAddress']
        if roadAddress is None:
            search_address = address['jibunsAddress']['list'][0]['fullAddress']
        else:
            search_address= address['roadAddress']['list'][0]['fullAddress']
            
        params3['address'] = search_address
        page=1
        params3['page'] = page
        time.sleep(random.uniform(1.5,2.5))
        #주위 장소 리스트 호출
        response2 = requests.get(url3,params3,headers=headers)
        if response2.status_code == 200:
            j_text2 = response2.json()
        elif response2.status_code == 404:
            print(f'{v} 주위장소 리스트 존재 X')
            none_lst2.append(v)
            continue
        else:
            print(f'{v} 주위장소 리스트 호출 실패')
            sys.exit('response2.status_code 조회해보기')
        max_page = 20
        total_pages = j_text2['place']['totalCount']
        while max_page * page < total_pages:
            page += 1
            params3['page'] = page
            lsts=j_text2['place']['list']
            for lst in lsts:
                cats = lst['category']
                for cat in cats:
                    #숙박업소 있는지 조회    
                    if '숙박' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(random.uniform(1.5,2.5))
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            stay_result.append(tmp_dict)
                            print(f'{v}의 숙박 {tmp_id}요약본 저장 완료')
                        else:
                            print(f'{v}로 숙박 요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
                    elif '호텔' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(random.uniform(1.5,2.5))
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            stay_result.append(tmp_dict)
                            print(f'{v} 중 {tmp_id}의  호텔요약본 저장 완료')
                        else:
                            print(f'{v}로 호텔요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
                    elif '리조트' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(random.uniform(1.5,2.5))
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            stay_result.append(tmp_dict)
                            print(f'{v} 중 {tmp_id}의  리조트요약본 저장 완료')
                        else:
                            print(f'{v}로 리조트요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
            #새로운페이지 불러옴
            time.sleep(2)
            response2 = requests.get(url3,params3,headers=headers)
            if response2.status_code == 200:
                j_text2 = response2.json()
                current_page = j_text2['place']['count']
            else:
                print(f'{v}의 장소 {page} 페이지 출력실패')
                sys.exit('response2.status_code 조회해보기')
        #while문 나온뒤 마지막 페이지 또는 최초 실행 구문
        lsts=j_text2['place']['list']
        for lst in lsts:
            cats = lst['category']
            for cat in cats:
                #숙박업소 있는지 조회    
                if '숙박' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(random.uniform(1.5,2.5))
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        stay_result.append(tmp_dict)
                        print(f'{v} 중 {tmp_id}의 숙박요약본 저장 완료')
                    else:
                        print(f'{v}로 여행지 숙박져오기 실패')
                        sys.exit('response3.status_code 조회해보기')
                elif '호텔' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(random.uniform(1.5,2.5))
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        stay_result.append(tmp_dict)
                        print(f'{v} 중 {tmp_id}의 호텔약본 저장 완료')
                    else:
                        print(f'{v}로 호텔 요약본 가져오기 실패')
                        sys.exit('response3.status_code 조회해보기')
                elif '리조트' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(random.uniform(1.5,2.5))
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        stay_result.append(tmp_dict)
                        print(f'{v} 중 {tmp_id}의 호텔약본 저장 완료')
                    else:
                        print(f'{v}로 호텔 요약본 가져오기 실패')
                        sys.exit('response3.status_code 조회해보기')                        
        print(f'{i}번째 {v}완료')

제주특별자치도 서귀포시 안덕면 신화역사로304번길 89 중 728951611의 숙박요약본 저장 완료
제주특별자치도 서귀포시 안덕면 신화역사로304번길 89 중 728951611의 호텔약본 저장 완료
0번째 제주특별자치도 서귀포시 안덕면 신화역사로304번길 89완료
제주특별자치도 제주시 구좌읍 월정1길 54-11 중 37915894의 숙박요약본 저장 완료
1번째 제주특별자치도 제주시 구좌읍 월정1길 54-11완료
제주특별자치도 제주시 한경면 저지12길 60-6 주위장소 리스트 존재 X
제주특별자치도 제주시 용마로4길 15 중 1587881986의 숙박요약본 저장 완료
3번째 제주특별자치도 제주시 용마로4길 15완료
제주특별자치도 제주시 한경면 신창상동길 20-1 중 32799937의 숙박요약본 저장 완료
4번째 제주특별자치도 제주시 한경면 신창상동길 20-1완료
제주특별자치도 제주시 구좌읍 하도15길 153-5 중 1885623509의 숙박요약본 저장 완료
5번째 제주특별자치도 제주시 구좌읍 하도15길 153-5완료
6번째 제주특별자치도 제주시 구좌읍 평대4길 20-1완료
제주특별자치도 서귀포시 안덕면 서광서문로 55-7 중 17083684의 숙박요약본 저장 완료
7번째 제주특별자치도 서귀포시 안덕면 서광서문로 55-7완료
8번째 제주특별자치도 제주시 관덕로13길 11완료
제주특별자치도 서귀포시 안덕면 난드르로21번길 13-7 중 1439489700의 숙박요약본 저장 완료
9번째 제주특별자치도 서귀포시 안덕면 난드르로21번길 13-7완료
10번째 제주특별자치도 서귀포시 남원읍 서성로 459완료
제주특별자치도 제주시 애월읍 유수암동3길 45 중 12947850의 숙박요약본 저장 완료
11번째 제주특별자치도 제주시 애월읍 유수암동3길 45완료
제주특별자치도 제주시 애월읍 하소로 769-15 중 11711598의 호텔약본 저장 완료
12번째 제주특별자치도 제주시 애월읍 하소로 769-15완료
제주특별자치도 제주시 애월읍 천덕로 313-3 중 1981161995의 숙박

96번째 제주특별자치도 서귀포시 표선면 표선당포로 9완료
제주특별자치도 제주시 한림읍 협재1길 27 중 1888386072의 숙박요약본 저장 완료
97번째 제주특별자치도 제주시 한림읍 협재1길 27완료
제주특별자치도 제주시 추자면 추자로 34 중 13217071의 숙박요약본 저장 완료
98번째 제주특별자치도 제주시 추자면 추자로 34완료
제주특별자치도 서귀포시 안덕면 화순중앙로 93 주위장소 리스트 존재 X
제주특별자치도 서귀포시 대정읍 일과대수로11번길 16-8 중 1297593807의 숙박요약본 저장 완료
100번째 제주특별자치도 서귀포시 대정읍 일과대수로11번길 16-8완료
제주특별자치도 제주시 한경면  고조로 311-8 중 1730959571의 숙박요약본 저장 완료
101번째 제주특별자치도 제주시 한경면  고조로 311-8완료
102번째 제주특별자치도 제주시 구좌읍 동복로 56-3완료
제주특별자치도 제주시 애월읍 고내로7길 10-2 중 1928178609의 숙박요약본 저장 완료
103번째 제주특별자치도 제주시 애월읍 고내로7길 10-2완료
제주특별자치도 서귀포시 효돈로204번길 23 주위장소 리스트 존재 X
105번째 제주특별자치도 서귀포시 성산읍 섭지코지로 62완료
제주특별자치도 제주시 구좌읍 세화11길 18 중 1270202089의 숙박요약본 저장 완료
106번째 제주특별자치도 제주시 구좌읍 세화11길 18완료
107번째 제주특별자치도 서귀포시 표선면 성읍민속로149번길 1완료


In [23]:
none_df2 =jeju_stay[jeju_stay['roadaddress'].isin(none_lst2)]

In [24]:
none_df2

,alltag,contentsid,contentscd,title,address,roadaddress,tag,introduction,readcnt,likecnt,...,evelpt,latitude,longitude,phoneno,sbst,PHOTOSHARECOUNT,cate2cd,cate3cd,imgpath,thumbnailpath
15,"민박,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,음료대,유도 및 안...",CNTS_000000000018758,숙박,UncleBORO게스트하우스,제주특별자치도 제주시 한경면 저지리 3263-5,제주특별자치도 제주시 한경면 저지12길 60-6,"민박,숙소,게스트하우스,농어촌민박,올레,산책로",조용한 소규모 여행자를 위한 정갈하고 여유로운 여자전용 게스트하우스,2177,0,...,3.0,33.338380,126.262535,070-8861-4343,??\r\n\r\n\r\n한경면 저지리에 위치한 엉클보로 게스트하우스는 여성전용 게...,1.0,게스트 하우스,게스트 하우스,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
225,"게스트하우스,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,음료대,유도...",CONT_000000000500847,숙박,라메종베니in제주,제주특별자치도 서귀포시 성산읍 고성리 188-1,제주특별자치도 서귀포시 성산읍 신양로122번길 4-6,"게스트하우스,펜션,휴양펜션,민박,마을관광,조식,농어촌민박",축복받은 집이라는 의미를 담고 있는 신양해수욕장과 섭지코지 앞 게스트하우스,1972,1,...,NaN,33.439795,126.899064,064-783-7818,라메종베니 게스트하우스는 관광명소인 섭지코지로 진입하는 마을 입구에 위치한 숙소이다...,NaN,게스트 하우스,게스트 하우스,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
230,"펜션,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,유도 및 안...",CNTS_000000000018921,숙박,해안길펜션,제주특별자치도 제주시 우도면 연평리 751,제주특별자치도 제주시 우도면 우도해안길 690,"펜션,숙소,휴양펜션,바비큐,교통,해수욕장,해변,공공와이파이존,독채,우도,단체여행객,...",우도 해안길에 위치한 펜션,1045,0,...,5.0,33.519740,126.958520,064-783-2087,??\r\n\r\n우도의 북서쪽 바닷가에 위치한 해안길 펜션은 바다에서 몰려오는 습...,1.0,펜션/민박,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
349,"모텔,현금결제,카드결제,무료 WIFI,흡연구역,편의점,음료대,유도 및 안내시설,경보...",CONT_000000000500953,숙박,신강남모텔,제주특별자치도 제주시 용담일동 2827-6,제주특별자치도 제주시 서광로3길 2,"모텔,숙소,공항,교통,공공와이파이존,주차장,편의시설",제주시외버스 터미널 바로 앞에 있는 모텔,2697,0,...,NaN,33.500424,126.513608,064-753-8770,??제주시 시외버스 종합터미널 맞은편에 위치하고 있어 제주도 전지역을 이동하는데 교...,NaN,모텔/여관,모텔,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
378,"휴식/힐링,숙소,호텔,공공와이파이존,음식,엘리베이터,관광호텔,공용주차장,현금결제,카...",CONT_000000000500948,숙박,스카이리더스호텔,제주특별자치도 제주시 연동 268-13,제주특별자치도 제주시 신광로 106,"휴식/힐링,숙소,호텔,공공와이파이존,음식,엘리베이터,관광호텔",제주시의 중심지에 위치한 호텔,1247,0,...,NaN,33.482616,126.489657,064-747-0003,제주특별자치도 지정 우수관광사업체이다. 공항에서 10분거리에 위치해 있다.\r\n바...,NaN,호텔,기타 호텔,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
489,"펜션,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,유도 및 안...",CNTS_000000000018681,숙박,스머프하우스,제주특별자치도 서귀포시 중문동 1489-3,제주특별자치도 서귀포시 천제연로 127-1,"펜션,숙소,휴양펜션,민박,바비큐,산책로,해수욕장,자연경관,해변","푸른바다와 산책로가 있는, 황토와 나무로 지어진 펜션",2673,0,...,NaN,33.254263,126.419214,064-738-9555,제주관광의 중심지인 중문관광단지 내에 위치하고 있으며 푸른바다가 펼쳐진 아름다운 산...,NaN,펜션/민박,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
490,"휴식 ,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,음료대,유도 및 ...",CONT_000000000500768,숙박,cafe말/개스트하우스 말,제주특별자치도 서귀포시 월평동 72-2,제주특별자치도 서귀포시 이어도로 450,"휴식,게스트하우스,민박,반려동물보호소,조식,카페",반려동물 동반 가능한 게스트하우스,1375,0,...,NaN,33.241654,126.467039,064-738-9397,??\r\n\r\n\r\n'개'스트하우스 '말'은 서귀포시 월평동에 위치한 게스트하...,NaN,게스트 하우스,게스트 하우스,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
505,"휴식 ,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,음료대,유...",CNTS_000000000019325,숙박,라오체리조트,제주특별자치도 서귀포시 강정동 852-1,제주특별자치도 서귀포시 이어도로 763 (강정동),"휴식,숙소,리조트,서귀포,휴양콘도,공공와이파이존,수영장,주차장","소각족이 이용하기 좋은 디럭스 객실, 친구들이나 가족이 이용하기좋은 스위트 객실, ...",2720,0,...,NaN,33.236057,126.496790,064-738-7575,조식(전복보말죽과 매실차)은 객실별 정원에 한해 무료로 제공해드립니다.(그랜드스위트...,NaN,리조트,리조트,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
511,"안전인증숙소,숙소,펜션,휴양펜션,우수관광사업체,자연경관,해변,중문관광단지,일몰명소,...",CONT_000000000501037,숙박,제주디오빌,제주특별자치도 서귀포시 색달동 2200,제주특별자치도 서귀포시 색달로72번길 20,"안전인증숙소,숙소,펜션,휴양펜션,우수관광사업체,자연경관,해변,중문관광단지,일몰명소,...",서귀포 중문관광단지 인근에 위치한 뷰가 아름다운 펜션,2142,0,...,NaN,33.259098,126.410590,064-738-6555,제주관광의 중심지 중문관광단지에서 5분거리에 위치해있다.시원하게 펼쳐지는 수평선과 ...,NaN,펜션/민박,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
527,"펜션,공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점

In [26]:
none_df2.shape

(25, 25)

In [27]:
result_df = pd.DataFrame(stay_result)

In [32]:
result_df.drop_duplicates(subset='id',inplace = True)

In [33]:
result_df.to_csv('../data/naver_crawling/stay_data_raw.csv',index=False)

In [36]:
result_df.shape

(1401, 61)